In [1]:
import pandas as pd

In [2]:
import json
import pandas as pd
data = json.load(open('dat.json'))

df = pd.DataFrame(data["results"])

In [4]:
df['seriousnessdeath']

0    NaN
1      1
2    NaN
3    NaN
4    NaN
Name: seriousnessdeath, dtype: object

In [6]:
#print(df['patient'].iloc[0].keys())
#print(df['patient'].iloc[0]['drug'][0]['medicinalproduct'])
for j in df['patient'].iloc[0]['drug']:
    print(j['medicinalproduct'])

OROKEN (CEFIXIME, UNSPEC)
PYOSTACINE (PRISTINAMYCIN)
ROCEPHIN
OFLOXACIN


In [11]:
print(df['patient'].iloc[0].keys())
for j in df['patient'].iloc[0]['reaction']:
    print(j)

dict_keys(['patientonsetage', 'reaction', 'patientonsetageunit', 'patientsex', 'drug'])
{'reactionmeddrapt': 'ARTHRALGIA'}
{'reactionmeddrapt': 'OEDEMA PERIPHERAL'}
{'reactionmeddrapt': 'PURPURA'}


In [3]:
print(df.head())

  receiptdateformat                                           receiver  \
0               102                                               None   
1               102                                               None   
2               102  {'receivertype': '6', 'receiverorganization': ...   
3               102  {'receivertype': '6', 'receiverorganization': ...   
4               102  {'receivertype': '6', 'receiverorganization': ...   

                companynumb receivedateformat  \
0          HQWYE821915MAR04               102   
1                JACAN16471               102   
2                   1289378               102   
3     US-JNJFOC-20130719067               102   
4  US-PFIZER INC-2014068976               102   

                                       primarysource seriousnessother  \
0                                               None                1   
1  {'qualification': '3', 'reportercountry': 'CAN...              NaN   
2    {'qualification': '5', 'reportercoun

In [10]:
print(df['occurcountry'])

0    NaN
1    NaN
2    NaN
3    NaN
4     US
Name: occurcountry, dtype: object


In [7]:
import requests

page = requests.get('https://api.fda.gov/drug/event.json?limit=100&skip=1')
dat=page.json()



In [17]:
print(dat)

{'meta': {'disclaimer': 'Do not rely on openFDA to make decisions regarding medical care. While we make every effort to ensure that data is accurate, you should assume all results are unvalidated. We may limit or otherwise restrict your access to the API in line with our Terms of Service.', 'terms': 'https://open.fda.gov/terms/', 'license': 'https://open.fda.gov/license/', 'last_updated': '2020-05-02', 'results': {'skip': 1, 'limit': 100, 'total': 11901829}}, 'results': [{'receiptdateformat': '102', 'receiver': None, 'seriousnessdeath': '1', 'companynumb': 'JACAN16471', 'receivedateformat': '102', 'primarysource': {'qualification': '3', 'reportercountry': 'CANADA'}, 'transmissiondateformat': '102', 'fulfillexpeditecriteria': '1', 'safetyreportid': '5801206-7', 'sender': {'senderorganization': 'FDA-Public Use'}, 'receivedate': '20080707', 'patient': {'patientonsetage': '26', 'patientdeath': {'patientdeathdate': None, 'patientdeathdateformat': None}, 'reaction': [{'reactionmeddrapt': 'DR

In [63]:
def chunk_preprocessing(json_file):
    df = pd.DataFrame(json_file["results"])
    #print(df)
    med_lst_global=[]
    react_lst_global=[]
    for ind in range(0,df.shape[0]):
        med_lst=[]
        react_lst=[]
        ser_lst=[]
        ser_death_lst=[]
        for medicine in df['patient'].iloc[ind]['drug']:
            med_lst.append(medicine['medicinalproduct'])
        for reaction in df['patient'].iloc[ind]['reaction']:
            react_lst.append(reaction['reactionmeddrapt'])
        med_lst_global.append(med_lst)
        react_lst_global.append(react_lst)
    meds=zip(med_lst_global)
    med=pd.DataFrame(meds)
    reacts=zip(react_lst_global)
    react=pd.DataFrame(reacts)
    ser=df['serious']
    disabling=df['seriousnessdisabling']
    death=df['seriousnessdeath']
    country=df['occurcountry']
    df_prep=pd.concat([med,react,ser,death,disabling,country],axis=1,ignore_index=True)
    df_prep.rename(columns={0:'medications',1:'adverse_reactions',2:'serious',3:'death',4:'disabling',5:'country'},inplace=True)
    flat_med_lst = []
    #Flatten the list 
    for n, row in df_prep.iterrows():
        for item in row['medications']:
            row['flat_meds'] = item
            flat_med_lst += [row.copy()]
    flat_med_df = pd.DataFrame(flat_med_lst)
    return(flat_med_df)
    

In [64]:
prepr=chunk_preprocessing(dat)
#df = pd.DataFrame(dat["results"])
print(prepr)
print(prepr.iloc[6])
print(prepr.medications.values)

                                          medications  \
0                                     [DURAGESIC-100]   
1                                            [BONIVA]   
2                                         [IBUPROFEN]   
3                                            [LYRICA]   
4   [DOXYCYCLINE HYCLATE, TRAMADOL HYDROCHLORIDE, ...   
..                                                ...   
97         [DONEPEZIL, DONEPEZIL (UNKNOWN), TRAMADOL]   
97         [DONEPEZIL, DONEPEZIL (UNKNOWN), TRAMADOL]   
97         [DONEPEZIL, DONEPEZIL (UNKNOWN), TRAMADOL]   
98                                         [LETAIRIS]   
99                                         [LETAIRIS]   

                                    adverse_reactions serious death disabling  \
0               [DRUG ADMINISTRATION ERROR, OVERDOSE]       1     1       NaN   
1         [Vomiting, Diarrhoea, Arthralgia, Headache]       1   NaN         1   
2                       [Dyspepsia, Renal impairment]       1   NaN     

In [26]:
import csv
response=requests.get('https://developers.google.com/public-data/docs/canonical/countries_csv')
print(page)

reader = csv.DictReader(response.iter_lines())
for record in reader:
    print(record)
#countries=page.csv()

c_df=pd.read_json(countries)

print(c_df)

<Response [200]>


Error: iterator should return strings, not bytes (did you open the file in text mode?)

In [35]:
df_=df['occurcountry']#.dropna()
#df.rename(columns={'occurcountry':'country_code'},inplace=True)
df_.columns=['countrycode']
print(df_.columns)
print(df_)

['countrycode']
0    NaN
1    NaN
2    NaN
3    NaN
4     US
Name: occurcountry, dtype: object


In [17]:
df = pd.DataFrame(dat["results"])

In [18]:
print(df['occurcountry'])

0     NaN
1     NaN
2     NaN
3     NaN
4      US
     ... 
95     US
96     US
97     IT
98     GB
99     US
Name: occurcountry, Length: 100, dtype: object


In [20]:
print(df)

   receiptdateformat                                           receiver  \
0                102                                               None   
1                102                                               None   
2                102  {'receivertype': '6', 'receiverorganization': ...   
3                102  {'receivertype': '6', 'receiverorganization': ...   
4                102  {'receivertype': '6', 'receiverorganization': ...   
..               ...                                                ...   
95               102  {'receivertype': '6', 'receiverorganization': ...   
96               102  {'receivertype': '6', 'receiverorganization': ...   
97               102  {'receivertype': '6', 'receiverorganization': ...   
98               102  {'receivertype': '6', 'receiverorganization': ...   
99               102  {'receivertype': '6', 'receiverorganization': ...   

                     companynumb receivedateformat  \
0               HQWYE821915MAR04             

In [21]:
df.to_csv('dat.csv')